<a href="https://colab.research.google.com/github/CS23M005/Assignment2_PartA/blob/main/CS23M005_A2_PartA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
import wandb
wandb.login()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, input_size, output_size, num_filters, filter_size, activation_fun, filter_config):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.reul1 = nn.relu()
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.fc1 = nn.Linear(64*4*4, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        # N, 3, 32, 32
        x = F.relu(self.conv1(x))   # -> N, 32, 30, 30
        x = self.pool(x)            # -> N, 32, 15, 15
        x = F.relu(self.conv2(x))   # -> N, 64, 13, 13
        x = self.pool(x)            # -> N, 64, 6, 6
        x = F.relu(self.conv3(x))   # -> N, 64, 4, 4
        x = torch.flatten(x, 1)     # -> N, 1024
        x = F.relu(self.fc1(x))     # -> N, 64
        x = self.fc2(x)             # -> N, 10
        return x


model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):

    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()

    print(f'[{epoch + 1}] loss: {running_loss / n_total_steps:.3f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
def train_nn():
    config_defaults = {
            "num_epochs": 3, \
            "num_hidden_layers":  4, \
            "size_hidden_layer": 128, \
            "learning_rate": 1e-3, \
            "optimizer": "rmsprop", \
            "batch_size": 32, \
            "weight_init": "xavier" , \
            "activation": "tanh", \
            "loss": "crossentropy", \
            "reg_lambda": 0.001, \
    }
    wandb.init(config = config_defaults, project = "Assignment1")
    config = wandb.config
    print(config.optimizer)
    wandb.run.name = "e_{}_hl_{}_opt_{}_bs_{}_init_{}_ac_{}_loss_{}".format(config.num_epochs,\
                                                                  config.num_hidden_layers,\
                                                                  config.size_hidden_layer,\
                                                                  config.learning_rate,\
                                                                  config.optimizer,\
                                                                  config.batch_size,\
                                                                  config.weight_init,\
                                                                  config.activation,\
                                                                  config.loss,\
                                                                  config.reg_lambda)

    seed(42)
    n_inputs = 784
    n_layers = config.num_hidden_layers
    n_neurons = config.size_hidden_layer
    n_outputs = 10
    init_mode = config.weight_init
    w,b = init_nn(n_inputs, n_layers, n_neurons, n_outputs, init_mode)
    w_old = w.copy()
    b_old = b.copy()

    eta = config.learning_rate
    activation_f = config.activation
    batch_size = config.batch_size
    loss = config.loss
    ud_lambda = config.reg_lambda
    optimizer = config.optimizer

    beta = 0.9
    beta1 = 0.9
    beta2 = 0.999

    eps = 1e-9
    max_epochs = config.num_epochs
    momentum = 0.9



    wandb.log({"examples": fig1})



    if optimizer == "sgd":
      w_n,b_n, a_n, h_n = sgd(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs)
    elif optimizer == "mgd":
      w_n,b_n, a_n, h_n = mgd(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, max_epochs, momentum)
    elif optimizer == "nag":
      w_n,b_n, a_n, h_n = nag(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs, momentum)
    elif optimizer == "rmsprop":
      w_n,b_n, a_n, h_n = rmsprop(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs, beta, eps)
    elif optimizer == "adam":
      w_n,b_n, a_n, h_n = adam(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs, beta1, beta2, eps)
    elif optimizer == "nadam":
      w_n,b_n, a_n, h_n = nadam(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs, beta1, beta2, eps)


    train_accuracy, yp = accuracy(X_train, y_train, w_n, b_n, n_layers, activation_f)
    validation_accuracy, yp = accuracy(X_validation, y_validation, w_n, b_n, n_layers, activation_f)
    test_accuracy, ytp = accuracy(X_test, y_test, w_n, b_n, n_layers, activation_f)

    cm = wandb.plot.confusion_matrix(y_true=y_test, preds=ytp, class_names=u_l)
    wandb.log({"conf_mat": cm})



    wandb.log({"train_acc": train_accuracy, \
               "validation_acc": validation_accuracy, \
                "test_acc": test_accuracy, \
                })
    # data = [[x, y] for (x, y) in zip(xxnn, trac)]
    # table = wandb.Table(data=data, columns = ["x", "y"])
    # wandb.log({"my_custom_plot_id" : wandb.plot.line(table,
    #                              "x", "y", title="train_accuracy_steps")})
    # data2 = [[x, y] for (x, y) in zip(xxnn, vaac)]
    # table2 = wandb.Table(data=data2, columns = ["x", "y"])
    # wandb.log({"my_custom_plot_id1" : wandb.plot.line(table2,
    #                              "x", "y", title="validation_accuracy_steps")})
    # data3 = [[x, y] for (x, y) in zip(xxnn, teac)]
    # table3 = wandb.Table(data=data3, columns = ["x", "y"])
    # wandb.log({"my_custom_plot_id2" : wandb.plot.line(table3,
    #                              "x", "y", title="test_accuracy_steps")})

    print("accu: ", validation_accuracy)




####################################################################

sweep_id = wandb.sweep(sweep_config, project = "Assignment1")
wandb.agent(sweep_id, function = train_nn, count=3)
wandb.finish()

#train_nn()
#wandb.finish()
